In [ ]:
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [157]:
#put all the relevant data into the notebook
current = pd.read_csv('/kaggle/input/fantasyfootball2019/p_current.csv') 
future = pd.read_csv('/kaggle/input/fantasyfootball2019/p_future.csv') 
yahoo = pd.read_csv('/kaggle/input/fantasyfootball2019/yahoo (2).csv') 
rosters = pd.read_csv('/kaggle/input/fantasyfootball2019/current_rosters.csv') 


In [ ]:
#find the yahoo projections (what everyone else sees) for every owned player in the league 
yahoo_rosters = pd.merge(rosters, yahoo,  how='inner', left_on=['first_name','last_name'], right_on = ['first_name','last_name'])
current_rosters = pd.merge(rosters, current,  how='inner', left_on=['first_name','last_name'], right_on = ['first_name','last_name'])
future_rosters = pd.merge(rosters, future,  how='inner', left_on=['first_name','last_name'], right_on = ['first_name','last_name'])

In [ ]:
# filter to only weighted values 
# (does not matter for yahoo, but will matter to extrapolate to other sources)
yahoo_weighted = yahoo_rosters.loc[yahoo_rosters['avg_type'] == 'weighted']
current_weighted = current_rosters.loc[current_rosters['avg_type'] == 'weighted']
future_weighted = future_rosters.loc[future_rosters['avg_type'] == 'weighted']

In [ ]:
# find avg points per position for league
# yahoo_weighted.select(dp, position, avg(floor), avg(avg_points), avg(ceiling).groupby(dp, postion))
starters_yahoo = yahoo_weighted[['dp', 'position_x', 'floor', 'ceiling']] \
                    .groupby(['dp', 'position_x']) \
                    .agg(['max'])
starters_tot = current_weighted[['dp', 'position_x', 'floor', 'ceiling']] \
                    .groupby(['dp', 'position_x']) \
                    .agg(['max'])

#TODO add rb2, wr2 and flex
starters_diff = (starters_tot - starters_yahoo).dropna().reset_index(inplace = True)

In [ ]:
# This should return our current team according to current_rosters input
current_weighted.where(current_weighted['dp'] == 2).dropna()

In [ ]:
ffpy = current_weighted.where(current_weighted['dp'] == 2).dropna()

In [ ]:
# Need mutually beneficial trades (prefereably more beneficial to them in yahoo and us in nonyahoo)
# when would our bench player start on someone else's team?

# Find our bench in standard (using ceiling points) 
# eliminate the top QB, 2 RB, 2 WR, TE, FLEX
qb_ffpy = ffpy.where(ffpy['position_y'] == 'QB').dropna()
rb_ffpy = ffpy.where(ffpy['position_y'] == 'RB').dropna()
wr_ffpy = ffpy.where(ffpy['position_y'] == 'WR').dropna()
te_ffpy = ffpy.where(ffpy['position_y'] == 'TE').dropna()
qb_starter = qb_ffpy[qb_ffpy['ceiling'] == qb_ffpy['ceiling'].max()]
rb_starter = rb_ffpy[rb_ffpy['ceiling'] == rb_ffpy['ceiling'].max()]
#rb2_starter
wr_starter = wr_ffpy[wr_ffpy['ceiling'] == wr_ffpy['ceiling'].max()]
#wr2_starter
te_starter = te_ffpy[te_ffpy['ceiling'] == te_ffpy['ceiling'].max()]
#flex_starter
bench_ffpy = ffpy.loc
# find our best bench player for each position in yahoo points

# find their starter with yahoo points

# compare our bench player with their least starter

In [ ]:
rb_starter